In [1]:
import sys
import import_ipynb
import numpy as np
from scipy.sparse.linalg import spsolve
from pathlib import Path

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps
    from pre_processing import create_domain, read_mesh
    from fem_processing import boundary_conditions
    from pos_processing import graph_results
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!
Modules imports were successful!
Modules imports were successful!


# Project 1: Poisson Problem at Rectangular Domain $\Omega = [0,1]^2$

Considere um problema de _Poisson_ 2D em $\Omega = [0,1]^2$

$$
-\nabla \cdot \left( \nabla u(x,y) \right) = f(x,y)
\tag{1}
$$

onde,
$$
f(x,y) = 2\pi^2 \, sin(\pi x) \, sin(\pi y)
\tag{2}
$$

cujas condições de contorno são $u=0$ em $\partial \Omega$.

A solução analítica deste problema é

$$
u(x,y) = sin(\pi x) \, sin(\pi y)
\tag{3}
$$

Implemente um programa de elementos finitos utilizando elementos triangulares $P_2$ para resolver o Problema de Valor de Contorno $(1)$.

# 9-noded Quadratic Quadrilateral Lagrangian Element $Q_2$
## Pre-processor module
### Geometry and mesh Domain

In [2]:
FINITE_ELEMENT = ("Quadrangle", 2)
BOUNDARY_CONDITIONS = [{'type': 'Dirichlet', 'tag': 101, 'value': 0.0, 'name': 'entire_boundary'}]
MATERIAL_PROPRIETIES = [{'type': 'material_constant_a', 'tag': 201, 'value': 1, 'name': 'entire_domain'}]

# Create mesh from file geometry
create_domain.rectangular_geometry(FINITE_ELEMENT, BOUNDARY_CONDITIONS, lc=0.05, view_mesh=False)
mesh_data = read_mesh.get_data(FINITE_ELEMENT, BOUNDARY_CONDITIONS, MATERIAL_PROPRIETIES, model='rectangular')
graph_results.plot_mesh(mesh_data, FINITE_ELEMENT, model='rectangular')

Model rectangular_domain_Quadrangle2 (2D)
Info     : 9 geometric entities
Info     : 2 Physical Groups
Info     : 1681 nodes in total
Info     : 400 2-D elements in total
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p11_rectangular_poisson\pre_processing\pictures\rectangular_meshed_domain_Quadrangle2.svg


### Figure 1: Rectangular meshed domain $\Omega=[0,1]^2$

<figure>
    <img src="pre_processing/pictures/rectangular_meshed_domain_Quadrangle2.svg" alt="Fig.1" style="width:100%;" />
    <figcaption>Figure 1: Rectangular meshed domain $\Omega=[0,1]^2$.</figcaption>
</figure>

In [3]:
nodes = [node['global_coord'] for node in mesh_data['nodes_data']]
conn = [element['conn_list'] for element in mesh_data['conn_data']]

#### `apply_physics()`

In [4]:
mesh_data = ps.apply_physics(mesh_data, FINITE_ELEMENT)
mesh_data['conn_data'][:1]

[{'TagID': 85,
  'conn_list': [1, 5, 161, 140, 24, 522, 523, 160, 524],
  'material': {'type': 'material_constant_a',
   'tag': 201,
   'value': 1,
   'name': 'entire_domain',
   'a_value': 1},
  'source': {'type': 'analytical_source_fx', 'value': 0.12151120784720292}}]

### Imposition of Boundary Conditions

In [5]:
dirichlet_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
dirichlet_values = [
    node['bc']['value'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
free_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Free']

print(f"The entire domain has {len(nodes)} nodes: " 
      f"{len(nodes) - len(dirichlet_nodes)} free nodes and {len(dirichlet_nodes)} Dirichlet nodes.")

The entire domain has 1681 nodes: 1521 free nodes and 160 Dirichlet nodes.


### Reduced Global matrix $A_{gr}$ with boundary conditions

In [6]:
Agr, fgr = boundary_conditions.get_reduced_global_matrix(mesh_data, FINITE_ELEMENT)
print("Reduced global matrix shape:", Agr.shape)

Reduced global matrix shape: (1521, 1521)


## Processor Module
### Reduced Global matrix

In [7]:
ur = spsolve(Agr.tocsr(), fgr.toarray())
potential_u = boundary_conditions.get_global_potentials(mesh_data, ur)

## Post-Processor

In [8]:
graph_results.approximate_fem_griddata(FINITE_ELEMENT, mesh_data, potential_u, model='rectangular')

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p11_rectangular_poisson\pos_processing\pictures\rectangular_domain_solution_Quadrangle2_griddata.svg


### Figure 3: Scalar Field Distribution

<figure>
    <img src="pos_processing/pictures/rectangular_domain_solution_Quadrangle2_griddata.svg" alt="Fig.3" style="width:100%;" />
    <figcaption>Figure 3: Post-Processor: Scalar Field Distribution and Triangular Mesh.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.